In [1]:
"""
BAW.PY
Implements the Barone-Adesi And Whaley model for the valuation of American options and their greeks. 
"""

import numpy as _np
import cmath as _cm

# Option Styles
_AMERICAN = 'American'
_EUROPEAN = 'European'
# Option Types 
_CALL = 'Call'
_PUT = 'Put'
# Output Types 
_VALUE = 'Value'
_DELTA = 'Delta'
_GAMMA = 'Gamma'
_VEGA = 'Vega'
_THETA = 'Theta'

_dS = 0.001
_dT = 1 / 365
_dV = 0.00001

_ITERATION_MAX_ERROR = 0.001

def _standardNormalPDF(x): 
    val = (1 / (2 * _cm.pi)**0.5) * _np.exp(-1 * (x**2) / 2)
    return val
def _standardNormalCDF(X):     
    y = _np.abs(X) 
    
    if y > 37: 
        return 0
    else:
        Exponential = _np.exp(-1 * (y**2) / 2)
        
    if y < 7.07106781186547:
        SumA = 0.0352624965998911 * y + 0.700383064443688
        SumA = SumA * y + 6.37396220353165
        SumA = SumA * y + 33.912866078383
        SumA = SumA * y + 112.079291497871
        SumA = SumA * y + 221.213596169931
        SumA = SumA * y + 220.206867912376
        SumB = 0.0883883476483184 * y + 1.75566716318264
        SumB = SumB * y + 16.064177579207
        SumB = SumB * y + 86.7807322029461
        SumB = SumB * y + 296.564248779674
        SumB = SumB * y + 637.333633378831
        SumB = SumB * y + 793.826512519948
        SumB = SumB * y + 440.413735824752
        _standardNormalCDF = Exponential * SumA / SumB
    else:
        SumA = y + 0.65
        SumA = y + 4 / SumA
        SumA = y + 3 / SumA
        SumA = y + 2 / SumA
        SumA = y + 1 / SumA
        _standardNormalCDF = Exponential / (SumA * 2.506628274631)
    
    if X > 0:
         return 1 - _standardNormalCDF
    else: 
         return _standardNormalCDF
        
def _priceEuropeanOption(option_type_flag, S, X, T, r, b, v):
    '''
    Black-Scholes
    '''

    d1 = (_np.log(S / X) + (b + v**2 / 2) * T) / (v * (T)**0.5)
    d2 = d1 - v * (T)**0.5

    if option_type_flag == 'Call':
        bsp = S * _np.exp((b - r) * T) * _standardNormalCDF(d1) - X * _np.exp(-r * T) * _standardNormalCDF(d2)
    else:
        bsp = X * _np.exp(-r * T) * _standardNormalCDF(-d2) - S * _np.exp((b - r) * T) * _standardNormalCDF(-d1)
        
    return bsp

def _priceAmericanOption(option_type_flag, S, X, T, r, b, v):
    '''
    Barone-Adesi-Whaley
    '''
    
    if option_type_flag == 'Call':
        return _approximateAmericanCall(S, X, T, r, b, v)
    elif option_type_flag == 'Put':
        return _approximateAmericanPut(S, X, T, r, b, v)
def _approximateAmericanCall(S, X, T, r, b, v):
    '''
    Barone-Adesi And Whaley
    '''

    if b >= r:
        return _priceEuropeanOption('Call', S, X, T, r, b, v)
    else:
        Sk = _Kc(X, T, r, b, v)
        N = 2 * b / v**2                                           
        k = 2 * r / (v**2 * (1 - _np.exp(-1 * r * T)))
        d1 = (_np.log(Sk / X) + (b + (v**2) / 2) * T) / (v * (T**0.5))
        Q2 = (-1 * (N - 1) + ((N - 1)**2 + 4 * k))**0.5 / 2
        a2 = (Sk / Q2) * (1 - _np.exp((b - r) * T) * _standardNormalCDF(d1))
        if S < Sk:
            return _priceEuropeanOption('Call', S, X, T, r, b, v) + a2 * (S / Sk)**Q2
        else:
            return S - X
def _approximateAmericanPut(S, X, T, r, b, v):
    '''
    Barone-Adesi-Whaley
    '''

    Sk = _Kp(X, T, r, b, v)
    N = 2 * b / v**2
    k = 2 * r / (v**2 * (1 - _np.exp(-1 * r * T)))
    d1 = (_np.log(Sk / X) + (b + (v**2) / 2) * T) / (v * (T)**0.5)
    Q1 = (-1 * (N - 1) - (((N - 1)**2 + 4 * k))**0.5) / 2
    a1 = -1 * (Sk / Q1) * (1 - _np.exp((b - r) * T) * _standardNormalCDF(-1 * d1))

    if S > Sk:
        return _priceEuropeanOption('Put', S, X, T, r, b, v) + a1 * (S / Sk)**Q1
    else:
        return X - S